<div class="alert alert-info
Yes, i see the same also, hope this time it'll be ok. i get once in a while a window where i have to click either 'cancel', 'overwrite' or 'reload'. i used 'overwrite' before. now the same but i downloaded the notebook and then submitted.
first time i have this thing; what i should click, 'reload'?
</div>

<center style='font-size:28px;'><u><b>ML in Business</b></u></center>

1. [Project Description](#start)
    * [Data Description](#dd)
2. [Imports](#imp)
3. [Preprocessing data](#pp)
    * [Pre-Processing Summation](#ppssm)
4. [Training a model for each region](#tmfer)
5. [Profitable reserve volume](#prvo)
6. [Profit Calculation](#prfclcl)
7. [Bootstrap](#boot)
8. [Conclusion](#conc)

# Project description <a id="start"></a>

You work for the OilyGiant mining company. Your task is to find the best place for a new well.
Steps to choose the location:
Collect the oil well parameters in the selected region: oil quality and volume of reserves;
Build a model for predicting the volume of reserves in the new wells;
Pick the oil wells with the highest estimated values;
Pick the region with the highest total profit for the selected oil wells.
You have data on oil samples from three regions. Parameters of each oil well in the region are already known. Build a model that will help to pick the region with the highest profit margin. Analyze potential profit and risks using the Bootstrapping technique.

## Description of the data <a id="dd"></a>

**Geo-Data**
- Features
<ul>id — unique oil well identifier</ul>
<ul>f0, f1, f2 — three features of points (their specific meaning is unimportant, but the features themselves are significant)</ul>
- Target
<ul>product — volume of reserves in the oil well (thousand barrels).</ul>

# Imports <a id="imp"></a>

In [1]:
#pip install -U scikit-learn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# Pre-Processing <a id="pp"></a>

In [3]:
try:
    geo_data_0 = pd.read_csv('geo_data_0.csv')
except:
    geo_data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')

try:
    geo_data_1 = pd.read_csv('geo_data_1.csv')
except:
    geo_data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    
try:
    geo_data_2 = pd.read_csv('geo_data_2.csv')
except:
    geo_data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [4]:
geo_data_0.info()
geo_data_1.info()
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [5]:
display(geo_data_0.head())
display(geo_data_1.head())
display(geo_data_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


No missing values.

In [6]:
geo_data_0 = geo_data_0.astype({'id':'string'})
geo_data_1 = geo_data_1.astype({'id':'string'})
geo_data_2 = geo_data_2.astype({'id':'string'})

In [7]:
geo_data_0.info()
geo_data_1.info()
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  string 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), string(1)
memory usage: 3.8 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  string 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), string(1)
memory usage: 3.8 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Co

In [8]:
print(geo_data_0.duplicated().sum())
print(geo_data_1.duplicated().sum())
print(geo_data_2.duplicated().sum())

0
0
0


no Duplicates. `Id` casted to string.

`id` column is kept in case specific wells should be analysed; ML-wise it has no importance and could else be dropped.
Linear Regression is the algorithm to be used for this data, so OHE and features scaling are to be done as well. As all relevant columns are numeric, that leaves only features to be scaled once data is split.

In [9]:
print('Area 0 -')
display(geo_data_0.describe().loc[:,'product'].to_frame())
print('Area 1 -')
display(geo_data_1.describe().loc[:,'product'].to_frame())
print('Area 2 -')
display(geo_data_2.describe().loc[:,'product'].to_frame())

Area 0 -


,product
count,100000.000000
mean,92.500000
std,44.288691
min,0.000000
25%,56.497507
50%,91.849972
75%,128.564089
max,185.364347


Area 1 -


,product
count,100000.000000
mean,68.825000
std,45.944423
min,0.000000
25%,26.953261
50%,57.085625
75%,107.813044
max,137.945408


Area 2 -


,product
count,100000.000000
mean,95.000000
std,44.749921
min,0.000000
25%,59.450441
50%,94.925613
75%,130.595027
max,190.029838


## Pre-Processing Summation <a id="ppssm"></a>

- Data is widely numeric, besides `id`. Feature scaling to be done.

- The `product` column of `geo_data_0` and `geo_data_2` is statistics-wise very similar. The mean of `geo_data_1` is lower by 30% relative to the pair's mean, while the std of all tables is pretty close.

# Training a model for each region <a id="tmfer"></a>

In [10]:
def dwer(data):
    data = data.drop('id',axis=1)
    target = data['product']
    features = data.drop('product',axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split\
    (features, target, test_size=0.4, random_state=12345)
    features_valid, features_test, target_valid, target_test = train_test_split\
    (features_valid, target_valid, test_size=0.5, random_state=12345)


    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = pd.DataFrame(scaler.transform(features_train))
    features_valid = pd.DataFrame(scaler.transform(features_valid))
    features_test = pd.DataFrame(scaler.transform(features_test))
    
    
    model = LinearRegression()
    model.fit(features_train,target_train)
    predicted_valid = pd.Series(model.predict(features_valid))
    print('Average volume of predicted reserves -',predicted_valid.mean())
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mse ** 0.5
    print("Model's RMSE:", rmse)
    
    return features_train, features_valid, features_test, target_train, target_valid, target_test, predicted_valid

features_train_0, features_valid_0, features_test_0, target_train_0, target_valid_0, target_test_0, predicted_0 = dwer(geo_data_0)
features_train_1, features_valid_1, features_test_1, target_train_1, target_valid_1, target_test_1, predicted_1 = dwer(geo_data_1)
features_train_2, features_valid_2, features_test_2, target_train_2, target_valid_2, target_test_2, predicted_2 = dwer(geo_data_2)

Average volume of predicted reserves - 92.5086249124955
Model's RMSE: 37.54151515850602
Average volume of predicted reserves - 68.49008336465836
Model's RMSE: 0.8858505980507572
Average volume of predicted reserves - 95.00076767282144
Model's RMSE: 39.72337541963802


<div>Predicted average volume results match the product's mean for each dataset. While all datasets have the same variance (as `describe` illustrated above), `geo_data_1` predictions have a substantially lower RMSE than the rest. It suggests a strong linear connection between `geo_data_1`'s features and target.</div>

# Profitable reserve volume <a id="prvo"></a>

The budget for development of 200 oil wells is 100 million USD. 1000 barrels (a unit of product) of raw material worth 4500 USD.

In [11]:
Budget_per_well = 500000 # 100M USD / 200 wells
val_per_product = 4500
critical_reserve = Budget_per_well / val_per_product
critical_reserve

111.11111111111111

111 is higher than all 3 regions predicted average volume reserves. Yet, there must be wells worth the effort as the mean is not so far, only 1/3 to 1 standard deviation away.

# Profit Calculation <a id="prfclcl"></a>

In [12]:
def profit(target,predictions,count=200):
    target = target.reset_index(drop=True)
    predictions = predictions.reset_index(drop=True)
    predictions_sorted = pd.Series(predictions, index=target.index).sort_values(ascending=False)
    top_target = target.loc[predictions_sorted.index][:count]
    profit = (top_target.sum() * val_per_product) - (count * Budget_per_well)
    return profit

In [13]:
tar10 = target_valid_0.head()
pre10 = predicted_0.head()
display(tar10)
display(pre10)

63713    156.180661
61461    136.863435
53670    183.095523
76953     54.254507
54117     82.135943
Name: product, dtype: float64

0    157.656975
1    105.816711
2    129.734594
3    105.190078
4    107.021106
dtype: float64

In [14]:
tar10res = tar10.reset_index(drop=True)
pre10res = pre10.reset_index(drop=True)
display(tar10res)
display(pre10res)

0    156.180661
1    136.863435
2    183.095523
3     54.254507
4     82.135943
Name: product, dtype: float64

0    157.656975
1    105.816711
2    129.734594
3    105.190078
4    107.021106
dtype: float64

In [15]:
predictions_sorted = pd.Series(pre10res, index=tar10res.index).sort_values(ascending=False)
predictions_sorted

0    157.656975
2    129.734594
4    107.021106
1    105.816711
3    105.190078
dtype: float64

order and indexes are OK

In [16]:
top_target = tar10res.loc[predictions_sorted.index][:3]
top_target

0    156.180661
2    183.095523
4     82.135943
Name: product, dtype: float64

order and indexes are ok, so function suppose to work. but numbers are the same as before

In [17]:
prf0, prf1, prf2 = profit(target_valid_0,predicted_0), profit(target_valid_1,predicted_1), profit(target_valid_2,predicted_2)    

print(f'Profit for region 0 top 200: {prf0}')
print(f'Profit for region 1 top 200: {prf1}')
print(f'Profit for region 2 top 200: {prf2}')

Profit for region 0 top 200: 32110105.274146125
Profit for region 1 top 200: 24150866.966815114
Profit for region 2 top 200: 26207105.23307593


In [18]:
print(f'Profit for region 0 top 200 assuming fixed product: -{(critical_reserve - predicted_0.mean()) * 200 * 4500}')
print(f'Profit for region 1 top 200 assuming fixed product: -{(critical_reserve - predicted_1.mean()) * 200 * 4500}')
print(f'Profit for region 2 top 200 assuming fixed product: -{(critical_reserve - predicted_2.mean()) * 200 * 4500}')

Profit for region 0 top 200 assuming fixed product: -16742237.578754049
Profit for region 1 top 200 assuming fixed product: -38358924.97180748
Profit for region 2 top 200 assuming fixed product: -14499309.094460707


Its witnessed that for regions 1 and 2, the profit from the top 200 wells is better compared to the profit that would have been obtained in case product was fixed, constant at the prediction's mean. Meaning, the loss of money is lower compared to the "fixed" case. It make sense, as 200 wells represent the top percent of the valid/test sets of 20000 length each.
<div style="color:red; size:5;">In region 0 its the opposite. Not sure how this is possible, maybe due to randomness? It had a strong resemblence to region 2 as witnessed above in the describe method, including similar Q1 and Q3 values. So its surprising the best 200 wells of region 0 are so poor, even compared with the predicted fixed-at-mean well product. Unexplained</div>

**Region 2 presumed to contain the best wells, as its loss is the lowest. As such, it has the best chance of including some set of profitable wells. Calculating profit only for the set of wells that contain more than `critical_reserve` of product for each region can show the optimal use of each region.**

In [19]:
def ideal_use(target,predictions,count = 200):
    
    predictions_sorted = pd.Series(predictions, index=target.index).sort_values(ascending=False)
    top_target = target.loc[predictions_sorted.index][:count]

    risk = 1 - (sum(top_target > critical_reserve) / count)
    more_than_crit = top_target[top_target > critical_reserve]
    profitable_wells = len(more_than_crit)
    ideal_profit = (more_than_crit.reset_index()['product'].sum() * val_per_product) - (len(more_than_crit) * Budget_per_well)
    print(f'Risk - {risk}')
    print(f'Number of profitable wells - {profitable_wells}')
    print(f'Ideal profit - {ideal_profit}')
print('Region 0')    
ideal_use(target_valid_0,predicted_0)
print()
print('Region 1') 
ideal_use(target_valid_1,predicted_1)
print()
print('Region 2')    
ideal_use(target_valid_2,predicted_2)

Region 0
Risk - 0.665
Number of profitable wells - 67
Ideal profit - 8094000.393640675

Region 1
Risk - 0.815
Number of profitable wells - 37
Ideal profit - 4124567.3098744117

Region 2
Risk - 0.6
Number of profitable wells - 80
Ideal profit - 11898739.538949922


As suspected, region 2 wins at all parameters.

# Bootstrap <a id="boot"></a>

In [20]:
def bootstrap_it(target, predictions, N_SAMP=500):
    target = target.reset_index(drop=True)
    predictions = predictions.reset_index(drop=True)
    state = np.random.RandomState(12345)
    values = []
    probs = pd.Series(predictions, index=target.index)
    for i in range(1000):
        target_subsample = target.sample(n=N_SAMP, replace=True, random_state=state)
        probs_subsample = probs[target_subsample.index]      
        values.append(profit(target_subsample,probs_subsample))
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    pos_prof = len(values[values > 0])
    risk = 1 - (pos_prof / len(values))
    boot_profit = values.mean()
    str_show1 = f'The average profit is {boot_profit:.0f}; Its confidence interval is between {lower:.0f} to {upper:.0f}; '
    str_show2 = f'The risk of losing money is {risk:.2f}'
    str_show = str_show1 + str_show2
    return boot_profit, lower, upper, risk, str_show

print('Region 0')
display(bootstrap_it(target_valid_0, predicted_0)[4])
print('Region 1')
display(bootstrap_it(target_valid_1, predicted_1)[4])
print('Region 2')
display(bootstrap_it(target_valid_2, predicted_2)[4])

Region 0


'The average profit is 3785466; Its confidence interval is between -1259955 to 9035494; The risk of losing money is 0.06'

Region 1


'The average profit is 4574662; Its confidence interval is between 260533 to 8559064; The risk of losing money is 0.02'

Region 2


'The average profit is 4126785; Its confidence interval is between -1098755 to 9095928; The risk of losing money is 0.06'

Region 1 is the winner, and anyway the only region to comply with risk criteria

# Conclusion <a id="conc"></a>

- Region 1 has the best results - highest mean, generally and when bootstrapped.
- Region 1 has the lowest risk